In [ ]:
import pandas as pd
from pyomo.environ import *

# Cargar datos (asegúrate de que df tiene las columnas correctas)
df = pd.read_csv('datos_talleres.csv')  # Reemplaza con tu archivo

# Definir insumos y productos
inputs = ['Staff', 'Show room space', 'Population in category 1', 'Population in category 2', 'Enquiries Alpha model', 'Enquiries Beta model']
outputs = ['Alpha sales', 'Beta sales', 'Profit']
DMUs = df['Garage Name'].tolist()

# Crear modelo para cada DMU
results = {}
for k in DMUs:
    model = ConcreteModel()
    
    # Variables de peso
    model.u = Var(outputs, domain=NonNegativeReals)
    model.v = Var(inputs, domain=NonNegativeReals)
    
    # Función objetivo (Maximizar la eficiencia del taller k)
    model.obj = Objective(
        expr=sum(df.loc[df['Garage Name'] == k, o].values[0] * model.u[o] for o in outputs),
        sense=maximize
    )
    
    # Restricciones de eficiencia para todos los talleres
    model.constraints = ConstraintList()
    for j in DMUs:
        model.constraints.add(
            sum(df.loc[df['Garage Name'] == j, o].values[0] * model.u[o] for o in outputs) 
            <= sum(df.loc[df['Garage Name'] == j, i].values[0] * model.v[i] for i in inputs)
        )
    
    # Normalización (denominador de la función de eficiencia = 1)
    model.norm = Constraint(
        expr=sum(df.loc[df['Garage Name'] == k, i].values[0] * model.v[i] for i in inputs) == 1
    )
    
    # Resolver
    solver = SolverFactory('glpk')  # Asegúrate de tener GLPK instalado
    solver.solve(model)
    
    # Guardar resultados
    results[k] = {
        'Efficiency': sum(df.loc[df['Garage Name'] == k, o].values[0] * model.u[o].value for o in outputs),
        'u': {o: model.u[o].value for o in outputs},
        'v': {i: model.v[i].value for i in inputs}
    }

# Convertir resultados en DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)
